In [1]:
import time
from Linear_sysmdl import System_Model_nti
from RTS_Smoother import RTS_Smoother_nti
from Linear_KF import KalmanFilter_nti
from RTS_NUV import RTS_UV_nti
import torch
from local_fit import LocalFitter, ExpWin_DS, moving_avg, LCR, merge
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
from LFKS import smooth_ecg_LFKS
from torch.distributions.normal import Normal
from gen_ecg import range_prec_r_dB
from ecg_tester import *


Running on the CPU


# "ECG" Signal
A ECG signal consists of 7 segments inside a cylce:
1. P wave (round)
2. PR segement
3. QRS complex:
    - negative Q wave
    - positive R wave
    - negative S wave
4. ST segement
5. T wave (round)

## Generate ECG

In [2]:
# number of traj
NB = 10

# load data to be filtered
ppath = 'Sim_ecg/' # parent path
traj_path = ppath + 'traj/' # traj path
Y_ALL = torch.load(traj_path+'ECG_noisy.pt')[4, 0:NB, :]
ecg_gt = torch.load(traj_path+'ECG_gt.pt')

T_ecg = ecg_gt.size(0)

# optimal window parameter
gamma_optm = [0.85, 0.7, 0.45, 0.2, 0.1]

# Smooting result of pure LF
Y_LF = torch.empty_like(Y_ALL)
# Smooting result of pure KS
Y_KS = torch.empty_like(Y_ALL)
# Smooting result of LF + KS
Y_LFKS = torch.empty_like(Y_ALL)

In [3]:
Y_LF.size()

torch.Size([10, 183])

KS

In [4]:
MSE_KS, std_MSE_KS, Y_KS = test_KS(NB, Y_ALL, ecg_gt, itr_r=60, itr_q=80, r_init=1.0, q_init=1.0, win_q=2)

KS on ECG - MSE:  tensor(-27.6104)  [dB]
Interence Time: 13.436274766921997  s


In [5]:
loss_fun = nn.MSELoss(reduction='mean')
MSEs_NF = torch.empty(size=[NB])
for j in range(0, NB):
    MSEs_NF[j] = loss_fun(Y_ALL[j,:], ecg_gt)

In [6]:
MSEs_NF

tensor([0.0009, 0.0010, 0.0008, 0.0011, 0.0011, 0.0011, 0.0010, 0.0009, 0.0010,
        0.0010])

In [7]:
10*torch.log10(MSEs_NF)

tensor([-30.3629, -30.0648, -30.7759, -29.5986, -29.4344, -29.4455, -29.8186,
        -30.3311, -29.8948, -30.1518])